# Random Walk Metrics


## Modules

In [28]:
import numpy as np
import pandas as pd
import math

from scipy.spatial import distance

from scipy.stats  import wrapcauchy
from scipy.stats import levy_stable

import plotly.graph_objects as go


## Class

In [19]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
    and also provides a bunch of high level functions
    """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## Activity 1 - Path length - (BM1 vs BM2 vs CRW)
    - Write a function that returns a Brownian Motion (BM) trajectory in pandas df.
    - Write a function that returns a Correlated Random Walk (CRW) trajectory in pandas df.
    - Write a function that returns the path length for a given trajectory.
    - Compare at least the path length of three trajectories as shown in the figure below.
    - Display the results using plotly.

In [32]:
n_steps = 1000
s_pos = [0,0]
speed = 3

trajectory_bw = pd.DataFrame(columns=['x_pos','y_pos'])
temp_df = pd.DataFrame([{'x_pos':s_pos[0], 'y_pos':s_pos[1]}])

trajectory_bw = pd.concat([trajectory_bw, temp_df], ignore_index=True)

velocity = Vec2d(speed, 0) # Starting position for velocity vector

for i in range(n_steps):
    turn_angle = np.random.choice([0, np.pi/2, np.pi, 3*np.pi/2])
    
    velocity = velocity.rotated(turn_angle)
    
    temp_df = pd.DataFrame([{'x_pos':trajectory_bw.x_pos[i] + velocity.x, 'y_pos':trajectory_bw.y_pos[i] + velocity.y}])
    trajectory_bw = pd.concat([trajectory_bw, temp_df], ignore_index=True)
    
    
fig_bw = go.Figure()

fig_bw.add_trace(go.Scatter(
    x=trajectory_bw.x_pos,
    y= trajectory_bw.y_pos,
    mode='lines',
    name='trajectory',
    showlegend=True
))

fig_bw.update_layout(
    title='Brownian Motion 2D',
)

In [36]:
n_steps = 1000
s_pos = [0,0]
speed = 5
coefficient = 0.4

r = wrapcauchy.rvs(loc=0, c=coefficient, size=n_steps)

velocity = Vec2d(speed, 0) # Velocity vector in starting position

trajectory_crw = pd.DataFrame(columns=['x_pos', 'y_pos'])
temp_df = pd.DataFrame([{'x_pos':s_pos[0],'y_pos':s_pos[1]}])

trajectory_crw = pd.concat([trajectory_crw,temp_df], ignore_index=True) 

for i in range(n_steps):
    turn_angle = r[i]
    
    velocity = velocity.rotated(turn_angle)
    
    temp_df = pd.DataFrame([{'x_pos':trajectory_crw.x_pos[i] + velocity.x, 'y_pos':trajectory_crw.y_pos[i] + velocity.y}])
    trajectory_crw = pd.concat([trajectory_crw,temp_df], ignore_index=True)
    
fig_crw = go.Figure()

fig_crw.add_trace(go.Scatter(
    x=trajectory_crw.x_pos,
    y=trajectory_crw.y_pos,
    mode='lines',
    name='CRW - trajectory',
    showlegend=True
))

fig_crw.update_layout(
    title='CRW 2D Trajectory'
)

In [40]:
dist = pd.DataFrame(columns=['bw_2d'])
temp_df = pd.DataFrame([{'bw_2d':s_pos[0]}])

dist = pd.concat([dist, temp_df], ignore_index=True)

for i in range(n_steps):
    temp_df = pd.DataFrame([{'bw_2d':dist.bw_2d[i] + distance.euclidean(trajectory_bw.iloc[i], trajectory_bw.iloc[i+1])}])
    dist = pd.concat([dist, temp_df], ignore_index=True)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x= dist.index,
    y= dist.bw_2d,
    name='BW 2d',
    mode='lines',
    showlegend=True
))

fig.update_layout(
    title='Distances'
)

fig.show()
    


,bw_2d
0,0
1,3.0
2,6.0
3,9.0
4,12.0
...,...
996,2988.0
997,2991.0
998,2994.0
999,2997.0
